# Annotations

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (19:12:11) Alles bereit 


In [2]:
# dfranks=get_all_word_rankings()
# dfranks

In [3]:
get_changepoints()

KeyError: "['selfdist_perc'] not in index"

In [3]:
ANNOCOLS = ['Changes','From','To','Notes','is_valid']

In [4]:
def start_sheet_changes(min_peaks=1):
    dfchangepoints=get_signif_novelty_scores(
        min_peaks=min_peaks
    ).sort_values(
        'foote_novelty_z',ascending=False
    ).drop_duplicates(
        'word',keep='first'
    ).set_index('word').join(
        get_all_novelty_scores().groupby('word').mean(),
        rsuffix='_avg_sign'
    ).join(
        get_all_novelty_scores().groupby('word').mean(),
        rsuffix='_avg'
    ).join(
        get_all_word_rankings()
    ).sort_values(
        'foote_novelty_z_avg_sign',
        ascending=False
    ).rename({'period':'changepoint'},axis=1)
    dfchangepoints['rank'] = dfchangepoints.foote_novelty_z_avg_sign.rank(
        ascending=False
    ).apply(int)
    
    for ac in ANNOCOLS: dfchangepoints[ac]=''
    prefcols=['rank','changepoint'] + ANNOCOLS + ['word_num_peaks','foote_novelty_z_avg_sign']
    dfchangepoints = dfchangepoints[
        prefcols + sorted([c for c in dfchangepoints.columns if c not in set(prefcols)])
    ]
    return round(dfchangepoints,2)

In [5]:
# odf=get_changeranks_sattelzeit().join(start_sheet_changes())
# odf

In [6]:
# read_sheet()

In [7]:
def update_sheet_changes(df=None):
    df=start_sheet_changes() if df is None else df
    df_now=read_sheet()
    if len(df_now):
        for col in ANNOCOLS:
            df[col]=df_now[col]
    df=df.fillna('')
    prefcols=['rank_word']
    sufcols=['lnm_perc_word','lnm_perc','lnm_z','lnm']
    pref=prefcols + ANNOCOLS + sufcols
    df=df[pref + [c for c in df.columns if c not in set(pref)]]
    df.rank_word=df.rank_word.apply(int)
    df=round(df,2)
    df=df.sort_values('rank_word')
    write_sheet(df)
    return df

In [8]:
# update_sheet_changes()

## Gather data from anno

In [10]:
# dfanno=read_sheet()

In [13]:
DFANNO=None
def get_dfanno(force=False,dbkey='dfanno'):
    global DFANNO
    print('Reading annotations from online')
    DFANNO=df=read_sheet()
    return DFANNO

In [ ]:
get_changepoints(force=True)

[Koselleck] (19:05:23) Computing... (+796.7s)
Counting rows: 100%|██████████| 15528/15528 [00:17<00:00, 906.03it/s] 


In [ ]:
dfanno=get_dfanno(force=False)
dfanno

In [20]:
def get_words_anno_valid():
    dfanno=get_dfanno()
    return dfanno[dfanno.is_valid.str.startswith('y')].index
def get_words_anno_invalid():
    dfanno=get_dfanno()
    return dfanno[dfanno.is_valid.str.startswith('n')].index
        

In [24]:
len(get_words_anno_valid())

77

In [42]:
def _respell_change(x):
    x=x.replace('z','s')
    x
    return x

def get_changes_anno(min_count=3):
    odf=pd.DataFrame([
        {'word':w, 'Change':cx.strip().title(), **dict(row)}
        for w,row in get_dfanno().query('Changes!=""').iterrows()
        for cx in row.Changes.split(',')
    ])
    odf.Change=odf.Change.apply(_respell_change)
    if min_count: odf=odf.groupby('Change').filter(lambda x: len(x)>=min_count)
    return odf

In [46]:
# get_changes_anno()